In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#================================================================#
#============ Preparacion de datos modelo baseline ==============#
#================================================================#

#====================== Import de librerias =====================#

import random
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from sklearn.preprocessing import LabelEncoder
import plotly.figure_factory as ff
import datetime
import plotly.express as px
import plotly.graph_objects as go
from datetime import date
import glob
import numpy as np
import torch
import pandas as pd
import numpy as np
import csv
import os
import scipy.sparse as sp
from typing import Tuple, Dict, Any, List
from tqdm import tqdm, trange
from IPython import embed
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn import preprocessing
from tqdm import tqdm
import random


# Definicion de hiperparametros
hparams = {
    'batch_size':64,
    'num_epochs':12,
    'hidden_size': 32,
    'learning_rate':1e-4,
}

# we select to work on GPU if it is available in the machine, otherwise will run on CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Nombre de columnas
col_names = {"col_id_product": "asin",
             "col_id_reviewer": "reviewerID",
             "col_unix_time": "timestamp",
             "col_year": "year",
             "col_rating": "overall"}

# Path de datos
path_data= "/content/drive/MyDrive/postgraduate/Trabajo_final/data"

# Numero de zero positions 
num_samples=199

#============ Definicion de funciones de extraccion y tratamiento ============#

def load_src_data(archivo):

  '''
  Funcion que importa datasets. 
  '''

  ### Import de información
  df = pd.read_pickle(archivo)

  return df

def preprocess_data(df, meta):  
  # Preprocesamos el dataset
  # Nos quedamos solo con usuario producto y time stamp
  # Pasamos ids a numerico
  # Los ids de producto y usuario no pueden ser los mismos, los transformamos para que el minimo id de producto se +1 el maximo de usuario

  data = df[[col_names["col_id_reviewer"], col_names["col_id_product"],  col_names["col_rating"],  col_names["col_unix_time"]]]

  # Aplicamos una lambda a la columna de rating para transformar los valores a 0 o 1
  data.loc[:, col_names["col_rating"]] = data[col_names["col_rating"]].apply(lambda x: 1 if x > 0 else 0)

  # Convertimos la columna de tiempo a formato fecha
  data.loc[:, col_names["col_unix_time"]] = pd.to_datetime(data[col_names["col_unix_time"]])

  # Codificación de etiquetas para el usuario
  le_usuario = preprocessing.LabelEncoder()
  le_usuario.fit(data[col_names["col_id_reviewer"]])
  data.loc[:, col_names["col_id_reviewer"]] = le_usuario.transform(data[col_names["col_id_reviewer"]]).astype('int32')

  # Codificación de etiquetas para el producto
  le_producto = preprocessing.LabelEncoder()
  le_producto.fit(data[col_names["col_id_product"]])
  data.loc[:, col_names["col_id_product"]] = le_producto.transform(data[col_names["col_id_product"]]).astype('int32')
  meta.loc[:, col_names["col_id_product"]] = le_producto.transform(meta[col_names["col_id_product"]]).astype('int32')

  add_dims = 0

  # Convertimos la data a formato numpy
  data = data.to_numpy()

  for i in range(data.shape[1] - 2):  # do not affect to timestamp
      # Hacemos que los valores empiecen desde 0
      data[:, i] -= np.min(data[:, i])
      # Re-indexamos
      data[:, i] += add_dims
      add_dims = np.max(data[:, i]) + 1

  dims_usuarios_productos = np.max(data[:,:2], axis=0) + 1

  meta.loc[:, col_names["col_id_product"]] = meta[col_names["col_id_product"]].apply(lambda x: x + dims_usuarios_productos[0]).astype('int32')


  print( "\n","Minimo id de usuario: ", np.min(data[:,0]), "\n",
        "Maximo id de usuario: ", np.max(data[:,0]), "\n",
        "Minimo id de producto: ", np.min(data[:,1]), "\n",
        "Maximo id de producto: ", np.max(data[:,1]), "\n",)
  
  return (data, meta, dims_usuarios_productos, le_producto)

def build_adj_mx(n_feat:int, data:np.ndarray) -> sp.dok_matrix :
    """
    Esta funcion construye una matriz de adyacencia a partir de los datos de entrada.
    La matriz de adyacencia es una representacion simetrica de las interacciones entre los usuarios y productos
    y también las interacciones entre productos y cualquier información adicional.

    :param n_feat: El número de características presentes en los datos de entrada (número de columnas)
    :param data: La matriz de datos de entrada, donde cada fila representa una interacción entre un usuario y un producto.
    :return: La matriz de adyacencia, representada como un objeto dok_matrix, que es una estructura eficiente para construir matrices sparse.
    """

    # Instanciamos el objeto train_mat como una dok_matrix
    # Una estructura eficiente para construir matrices sparse
    train_mat = sp.dok_matrix((n_feat, n_feat), dtype=np.float32)
    for x in tqdm(data, desc=f"BUILDING ADJACENCY MATRIX..."):
        # rellanamos la matriz, al ser simetrica rellenamos primero con logica usuario-producto y luego con logica producto-usuario
        train_mat[x[0], x[1]] = 1.0
        train_mat[x[1], x[0]] = 1.0
        # IDEA: Tratamos las caracteristicas que no son usuario o producto de forma diferente porque no consideramos
        #  las interacciones entre contextos
        # Añadimos informacion extra a parte de la interacion usuario producto (aqui podria ir el rating)
        if data.shape[1] > 2:
            for idx in range(len(x[2:])):
                train_mat[x[0], x[2 + idx]] = 1.0
                train_mat[x[1], x[2 + idx]] = 1.0
                train_mat[x[2 + idx], x[0]] = 1.0
                train_mat[x[2 + idx], x[1]] = 1.0

    return train_mat


def ng_sample(data: np.ndarray, dims: list, num_ng:int=4) -> Tuple[np.ndarray, sp.dok_matrix]:
  """
  Crea una matriz de interacciones de usuario-producto (rating) y aplica un muestreo negativo.
  
  Args:
  data: np.ndarray, con las interacciones usuario-producto.
  dims: lista, con los valores mínimo y máximo de los productos y usuarios.
  num_ng: int, número de interacciones negativas que se quieren generar por cada interacción positiva.
  
  Returns:
  np.ndarray, con las interacciones usuario-producto incluyendo las interacciones negativas generadas.
  sp.dok_matrix, con la matriz de interacciones usuario-producto.
  """

  # Creación de la matriz de interacciones positivas
  rating_mat = build_adj_mx(dims[-1], data)
  interactions = []
  min_item, max_item = dims[0], dims[1]
  for num, x in tqdm(enumerate(data), desc='Performando muestreo negativo...'):
      # Añade la interacción positiva al arreglo de interacciones
      interactions.append(np.append(x, 1))
      for t in range(num_ng):
          # Selecciona un producto al azar para generar una interacción negativa
          j = np.random.randint(min_item, max_item)
          # Verifica que la interacción no sea una interacción positiva previa
          while (x[0], j) in rating_mat or j == int(x[1]):
              j = np.random.randint(min_item, max_item)
          # Añade la interacción negativa al arreglo de interacciones
          interactions.append(np.concatenate([[x[0], j], x[2:], [0]]))
  # Devuelve la matriz con todas las interacciones y la matriz dispersa con las interacciones positivas
  return np.vstack(interactions), rating_mat


def create_test_no_interactions(train_x: np.ndarray, test_x: np.ndarray, dims_usuarios_productos: Tuple[int, int],  num_samples: int) -> np.ndarray:
    """
    Esta funcion se encarga de crear de manera eficiente un dataset que contenga las interacciones usuario-producto en test que no se hayan producido.
    
    Argumentos:
        train_x (np.ndarray): matriz de entrenamiento con las interacciones usuario-producto previas
        test_x (np.ndarray): matriz de prueba con las interacciones usuario-producto previas
        dims_usuarios_productos (Tuple[int, int]): rango de productos y usuarios disponibles
    
    Retorno:
        np.ndarray: una matriz con todas las interacciones usuario-producto en test que no se hayan producido
    """
    from tqdm import tqdm
    import random
    
    # Identificamos los usuarios presentes en la prueba
    usuarios_test = np.unique(test_x[:, 0])
    # Identificamos el rango de productos disponibles
    total_productos = range(dims_usuarios_productos[0]-1, dims_usuarios_productos[1])
    
    # Definimos la matriz solucion y rellenamos
    zero_positions = np.zeros((num_samples * len(usuarios_test), 2))
    start_index = 0

    # Recorremos cada usuario presente en la prueba
    for usuario in tqdm(usuarios_test):
        # Identificamos los productos en los que el usuario ha interactuado previamente en entrenamiento
        productos_train = np.unique(train_x[train_x[:, 0] == usuario][:, 1])
        productos_test = np.unique(test_x[test_x[:, 0] == usuario][:, 1])
        productos = np.concatenate((productos_train, productos_test))
        # Seleccionamos al azar 199 productos con los que el usuario no ha interactuado previamente
        productos_a_machear = random.choices(list(set(total_productos) - set(productos)), k=num_samples)
        
        # Asignamos los índices de inicio y fin para los ceros que representan la interacción usuario-producto
        end_index = start_index + num_samples
        zero_positions[start_index:end_index, 0] = usuario
        zero_positions[start_index:end_index, 1] = productos_a_machear
        start_index = end_index
            
    return zero_positions

def create_test_no_interactions_stratified_old(df: pd.DataFrame,
                                           test_x: np.ndarray,
                                           train_x: np.ndarray, 
                                           dims_usuarios_productos: Tuple[int, int], 
                                           num_estratos: int,
                                           num_samples: int,
                                           le_producto) -> np.ndarray:

  """
  Función: create_test_no_interactions_stratified

  Argumentos:
  - df: un objeto de tipo DataFrame de pandas que contiene las interacciones usuario-producto en entrenamiento.
  - test_x: un objeto de tipo ndarray de numpy que contiene las interacciones usuario-producto de prueba.
  - train_x: un objeto de tipo ndarray de numpy que contiene las interacciones usuario-producto en entrenamiento.
  - dims_usuarios_productos: una tupla de enteros que contiene el número de usuarios y productos en el conjunto de datos.
  - num_estratos: un entero que indica el número de estratos en los que dividir los productos según su popularidad.
  - num_samples: un entero que indica el número de muestras aleatorias que se tomarán de cada estrato para cada usuario.
  - le_producto: un objeto de tipo LabelEncoder de scikit-learn que se usa para transformar los IDs de los productos.

  Salida:
  - Un objeto de tipo ndarray de numpy que representa la matriz de interacciones usuario-producto para aquellos productos que el usuario no ha interactuado previamente. Cada fila representa una interacción usuario-producto y los valores de cada columna representan el id del usuario y el id del producto, respectivamente.

  """

  # Definicion de mensaje inicial
  mensaje_inicial="Iniciando muestreo estratificado basado en popularidad"
  print(mensaje_inicial)
  print("-"*len(mensaje_inicial), "\n")

  # Identificamos los usuarios presentes en test
  usuarios_test = np.unique(test_x[:, 0])

  # Identificamos el rango de productos disponibles
  total_productos = range(dims_usuarios_productos[0]-1, dims_usuarios_productos[1])

  # Calculamos la probabilidad como el numero de reviews por producto
  product_popularity = df.groupby('asin').size().reset_index(name='popularity')
  product_popularity["asin"]=le_producto.transform(product_popularity["asin"])
  product_popularity.loc[:, col_names["col_id_product"]] = product_popularity[col_names["col_id_product"]].apply(lambda x: x + dims_usuarios_productos[0]).astype('int32')

  # Definimos la funcion de densidad de donde vamos a samplear:
  # Create the density plot
  show_graph=True
  if show_graph==True:
    fig = ff.create_distplot([product_popularity['popularity']], ['Popularity'])
    fig.show()
  # Definimos el numero de estratos en los que dividir a os productos segun popularidad
  #num_estratos = 5
  product_popularity['stratum'] = pd.qcut(product_popularity['popularity'], num_estratos, labels=False)

  # Ordenar los estratos por popularidad media
  stratum_popularity = product_popularity.groupby('stratum')['popularity'].mean()
  stratum_popularity = stratum_popularity.sort_values(ascending=False)

  # Calcular las probabilidades de muestreo. Esta funcion puede ser definida de otra manera
  stratum_probs = stratum_popularity / stratum_popularity.sum()
  stratum_probs = stratum_probs.reset_index(drop=True)

  print("Probabilidad de los estratos: ")
  print( stratum_probs, "\n")

  # Definicion de sample por estrato
  #num_samples = 199
  stratum_test_sizes = (stratum_probs * num_samples)
  stratum_test_sizes = np.round(stratum_test_sizes).astype(int)

  # Si la suma de los tamaños de los estratos es mayor que num_samples, restamos uno al tamaño del estrato más grande
  while stratum_test_sizes.sum() > num_samples:
      stratum_test_sizes[stratum_test_sizes.argmin()] -= 1

  # Si la suma de los tamaños de los estratos es menor que num_samples, sumamos uno al tamaño del estrato más grande
  while stratum_test_sizes.sum() < num_samples:
      stratum_test_sizes[stratum_test_sizes.argmax()] += 1

  print("Tamaño de muestreo de los estratos de los estratos: " )
  print( stratum_test_sizes, "\n")

  # Definimos la matriz solucion y rellenamos
  zero_positions = np.zeros((num_samples * len(usuarios_test), 2))
  start_index = 0

  # Iteramos sobre los usuarios de prueba
  for usuario in tqdm(usuarios_test):
    
    # Iteramos sobre cada estrato y su tamaño correspondiente
    for stratum, size in stratum_test_sizes.items():
      
      # Identificamos los productos en los que el usuario ha interactuado previamente en entrenamiento y el producto de gt
      productos_train = np.unique(train_x[train_x[:, 0] == usuario][:, 1])
      productos_test = np.unique(test_x[test_x[:, 0] == usuario][:, 1])
      productos = np.concatenate((productos_train, productos_test))

      # Identificamos los productos del estrato actual
      products_in_stratum = product_popularity[product_popularity['stratum'] == stratum]['asin']

      # Filtramos los productos del estrato actual que el usuario ha interactuado previamente
      products_in_stratum_filtered = np.setdiff1d(products_in_stratum, productos)

      # Tomamos una muestra aleatoria sin reemplazo de los productos filtrados del estrato actual
      sampled_products = np.random.choice(products_in_stratum_filtered, size, replace=False)

      # Asignamos los índices de inicio y fin para los ceros que representan la interacción usuario-producto
      end_index = start_index + size
      zero_positions[start_index:end_index, 0] = usuario
      zero_positions[start_index:end_index, 1] = sampled_products
      start_index = end_index

  # Convertimos los valores de cero posiciones a entero
  zero_positions = zero_positions.astype(int)

  return(zero_positions)


def create_test_no_interactions_stratified(df: pd.DataFrame,
                                           test_x: np.ndarray,
                                           train_x: np.ndarray, 
                                           num_samples: int,
                                           le_producto: LabelEncoder,
                                           show_graph=False) -> np.ndarray:

  """
  Crea un conjunto de datos de prueba con interacciones negativas  basadas en la popularidad de los productos.

  Parameters
  ----------
  df: pandas.DataFrame
      Dataframe con columnas ['reviewerID', 'asin'] que representa las interacciones de los usuarios con los productos.
  test_x: numpy.ndarray
      Matriz de interacciones usuario-producto que se va a usar como referencia para generar interacciones negativas.
  train_x: numpy.ndarray
      Matriz de interacciones usuario-producto que representa las interacciones previas del usuario en el conjunto de entrenamiento.
  num_samples: int
      Numero de interacciones negativas por usuario.
  le_producto: sklearn.preprocessing.LabelEncoder
      Objeto LabelEncoder que se utiliza para transformar los IDs de producto a valores numéricos.
  show_graph: bool
      Si es True, muestra un gráfico de la distribución de popularidad de los productos.

  Returns
  -------
  numpy.ndarray
      Matriz de interacciones negativas para los usuarios de test.
  """


  # Definicion de mensaje inicial
  mensaje_inicial="Iniciando muestreo estratificado basado en popularidad"
  print(mensaje_inicial)
  print("-"*len(mensaje_inicial), "\n")

  # Identificamos los usuarios presentes en test
  usuarios_test = np.unique(test_x[:, 0])

  # Identificamos el rango de productos disponibles
  total_productos = range(dims_usuarios_productos[0]-1, dims_usuarios_productos[1])

  # Calculamos la probabilidad como el numero de reviews por producto
  product_popularity = df.groupby('asin').size().reset_index(name='popularity')
  product_popularity["asin"]=le_producto.transform(product_popularity["asin"])
  product_popularity.loc[:, col_names["col_id_product"]] = product_popularity[col_names["col_id_product"]].apply(lambda x: x + dims_usuarios_productos[0]).astype('int32')

  # Definimos la funcion de densidad de donde vamos a samplear:
  # Create the density plot
  if show_graph==True:
    print("De esta grafica de densidad vamos a samplear:", "\n")
    fig = ff.create_distplot([product_popularity['popularity']], ['Popularity'])
    # Añade título al eje x
    fig.update_xaxes(title_text="Productos con X reviews")

    # Añade título al eje y
    fig.update_yaxes(title_text="Probabilidad")

    # Añade título al gráfico
    fig.update_layout(title="Grafico de densidad de popularidad")
    fig.show()
  
  # Genera un array con los valores de popularidad
  popularity_values = np.array(product_popularity['popularity'])

  # Calcula la probabilidad de cada valor de popularidad
  counts = pd.Series(popularity_values).value_counts()
  probabilities = counts / counts.sum()

  # Crea el dataframe con las columnas de popularidad y probabilidad
  density_df = pd.DataFrame({'popularity': probabilities.index, 'probability': probabilities.values})

  product_popularity=product_popularity.merge(density_df, how="left")

  # Definimos la matriz solucion y rellenamos
  zero_positions = np.zeros((num_samples * len(usuarios_test), 2))
  start_index = 0

  # Iteramos sobre los usuarios de prueba
  for usuario in tqdm(usuarios_test):

    # Identificamos los productos en los que el usuario ha interactuado previamente en entrenamiento y el producto de gt
    productos_train = np.unique(train_x[train_x[:, 0] == usuario][:, 1])
    productos_test = np.unique(test_x[test_x[:, 0] == usuario][:, 1])
    productos = np.concatenate((productos_train, productos_test))
  
    # Filtramos los productos que el usuario ha interactuado previamente
    filter_bool = product_popularity['asin'].isin(productos)
    filtered_df = product_popularity[~filter_bool]

    # Tomamos una muestra aleatoria sin reemplazo de los productos filtrados del estrato actual
    sampled_probabilities = filtered_df['probability'] / filtered_df['probability'].sum()
    sampled_products = np.random.choice(filtered_df['asin'], size=num_samples, p=sampled_probabilities)

    # Asignamos los índices de inicio y fin para los ceros que representan la interacción usuario-producto
    end_index = start_index + num_samples
    zero_positions[start_index:end_index, 0] = usuario
    zero_positions[start_index:end_index, 1] = sampled_products
    start_index = end_index

  # Convertimos los valores de cero posiciones a entero
  zero_positions = zero_positions.astype(int)

  return(zero_positions)




def build_test_set(itemsnoninteracted:list,
                   gt_test_interactions: np.ndarray) -> list:
    """
    Construye el conjunto de pruebas para un usuario dado
    :param itemsnoninteracted: lista de items que no han sido interactuados por el usuario
    :param gt_test_interactions: arreglo numpy con las interacciones verdaderas del usuario para pruebas
    :return: lista de arreglos numpy con interacciones positivas y negativas para cada usuario
    """
    # max_users, max_items = dims 
    test_set = []
    # Recorre cada par de interacciones verdaderas y items no interactuados
    for pair, negatives in tqdm(zip(gt_test_interactions, itemsnoninteracted), desc="BUILDING TEST SET..."):
        # AGREGA EL CONJUNTO DE PRUEBAS PARA UN SOLO USUARIO
        # Elimina el item que si fue interactuado
        negatives = np.delete(negatives, np.where(negatives == pair[1]))
        # Crea una matriz con la interacción verdadera y items no interactuados
        single_user_test_set = np.vstack([pair, ] * (len(negatives)+1))
        single_user_test_set[:, 1][1:] = negatives
        test_set.append(single_user_test_set.copy())
    return test_set



class PointData(Dataset):
    def __init__(self,
                 data: np.ndarray,
                 dims: list) -> None:
        """
        Dataset formatter adapted point-wise algorithms
        Parameters
        """
        super(PointData, self).__init__()
        self.interactions = data
        self.dims = dims

    def __len__(self) -> int:
        return len(self.interactions)
        
    def __getitem__(self, index: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Return the pairs user-item and the target.
        """
        return self.interactions[index][:-1], self.interactions[index][-1]

def split_train_test(data: np.ndarray,n_users: int) -> Tuple[np.ndarray, np.ndarray]:
    # Split and remove timestamp
    train_x, test_x = [], []
    for u in trange(n_users, desc='spliting train/test and removing timestamp...'):
        # Filtramos al usuario dentro del dataset
        user_data = data[data[:, 0] == u]
        # Ordenamos por fecha de interaccion
        sorted_data = user_data[user_data[:, 2].argsort()]

        # Si solo existe una review, introduce esta interaccion en el train
        if len(sorted_data) == 1:
            train_x.append(sorted_data[0][:-1])

        else:
          # Si la ultima interaccion tiene score de 1 puede ir a test, si no, todo a train
          if sorted_data[-1][-2]==1:
            # Introduce todas las interacciones salvo la ultima en el train
            train_x.append(sorted_data[:-1][:, :-1])
            # La ultima interaccion corresponde al test
            test_x.append(sorted_data[-1][:-1])

          else:
            train_x.append(sorted_data[:, :-1])

    return np.vstack(train_x), np.stack(test_x)


#============ Definicion de modelado ============#

#============ Definicion de visualizacion ============#

In [3]:
# Lectura de la información
file_to_src = glob.glob(path_data+'/SRC*.pkl')
df=load_src_data(file_to_src[0])# Reviews
meta=load_src_data(file_to_src[1])# Metadatos

In [4]:
# Preprocesamiento de reviews y metadatos
data, meta, dims_usuarios_productos, le_producto = preprocess_data(df, meta)


 Minimo id de usuario:  0 
 Maximo id de usuario:  10758 
 Minimo id de producto:  10759 
 Maximo id de producto:  14116 



In [5]:
# Division entre train y test
train_x, test_x = split_train_test(data, dims_usuarios_productos[0])

spliting train/test and removing timestamp...: 100%|██████████| 10759/10759 [00:32<00:00, 332.16it/s]


In [6]:
train_x=train_x.astype(int)
test_x=test_x.astype(int)

In [7]:
# Eliminamos timestamp
data=data[:,:-1]

In [8]:
# realizamos el negative sampling
train_x, rating_mat = ng_sample(train_x[:, :2], dims_usuarios_productos)

BUILDING ADJACENCY MATRIX...: 100%|██████████| 91008/91008 [00:02<00:00, 42022.12it/s]
Performando muestreo negativo...: 91008it [00:03, 24140.25it/s]


In [9]:
train_x

array([[    0, 10824,     1],
       [    0, 12358,     0],
       [    0, 11936,     0],
       ...,
       [10758, 10899,     0],
       [10758, 13829,     0],
       [10758, 12828,     0]])

In [10]:
# Creamos el dataset de entrenamiento. Devuelve array con interaccion usuario-producto y si sucedió de verdad
train_dataset = PointData(train_x, dims_usuarios_productos)
train_dataset[1]

(array([    0, 12358]), 0)

In [11]:
# Se crean interacciones que no se han dado (0's) en datos de test atendiendo a la popularidad
zero_positions=create_test_no_interactions_stratified(df,test_x,train_x,num_samples=num_samples,le_producto=le_producto, show_graph=True)

Iniciando muestreo estratificado basado en popularidad
------------------------------------------------------ 

De esta grafica de densidad vamos a samplear: 



100%|██████████| 10759/10759 [00:32<00:00, 328.96it/s]


In [12]:
# Se crean interacciones que no se han dado (0's) en datos de test de manera random
#zero_positions=create_test_no_interactions(train_x,test_x, dims_usuarios_productos, num_samples )

In [13]:
zero_positions

array([[    0, 11125],
       [    0, 12274],
       [    0, 14104],
       ...,
       [10758, 13371],
       [10758, 12719],
       [10758, 12757]])

In [14]:
# Se crea una lista de listas con los productos por usuario a testear
items2compute = []
for user in trange(dims_usuarios_productos[0]):
    aux = zero_positions[zero_positions[:, 0] == user][:, 1]
    items2compute.append(aux[aux >= dims_usuarios_productos[0]])

100%|██████████| 10759/10759 [01:27<00:00, 123.32it/s]


In [15]:
print(test_x)

[[    0 12806     1]
 [    1 14064     1]
 [    2 12659     1]
 ...
 [10756 12639     1]
 [10757 11148     1]
 [10758 14067     1]]


In [16]:
# Al estas en esta tabla todas las interacciones positivas, no necesito mas a esa ultima columna
test_x = test_x[:, :2]
test_x

array([[    0, 12806],
       [    1, 14064],
       [    2, 12659],
       ...,
       [10756, 12639],
       [10757, 11148],
       [10758, 14067]])

In [17]:
# Se construye el test_set definitivo, con las interacciones que han ocurrido y las que no en test set
test_x = build_test_set(items2compute, test_x)

BUILDING TEST SET...: 10759it [00:02, 5103.42it/s]


In [18]:
train_dataset.dims

array([10759, 14117], dtype=object)

In [19]:
test_x[0]

array([[    0, 12806],
       [    0, 11125],
       [    0, 12274],
       [    0, 14104],
       [    0, 11889],
       [    0, 12299],
       [    0, 12009],
       [    0, 11945],
       [    0, 13918],
       [    0, 13093],
       [    0, 13698],
       [    0, 11191],
       [    0, 10906],
       [    0, 13834],
       [    0, 11533],
       [    0, 11131],
       [    0, 13800],
       [    0, 10991],
       [    0, 11887],
       [    0, 13882],
       [    0, 11739],
       [    0, 11737],
       [    0, 13158],
       [    0, 11370],
       [    0, 12031],
       [    0, 12425],
       [    0, 11416],
       [    0, 11367],
       [    0, 11233],
       [    0, 13124],
       [    0, 11517],
       [    0, 13502],
       [    0, 12236],
       [    0, 13295],
       [    0, 13460],
       [    0, 14015],
       [    0, 13620],
       [    0, 13555],
       [    0, 12297],
       [    0, 12425],
       [    0, 12861],
       [    0, 11540],
       [    0, 13260],
       [   

In [20]:
# Guarda train_dataset y test_x
import pickle
from datetime import date

today = date.today()

# Train
with open(path_data+"/"+"MOD_baseline_train"+"_"+today.strftime("%d%m%Y")+".pkl", 'wb') as handle:
    pickle.dump(train_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Test
with open(path_data+"/"+"MOD_baseline_test"+"_"+today.strftime("%d%m%Y")+".pkl", 'wb') as f:
    pickle.dump(test_x, f)

# Meta
meta.to_pickle(path_data+"/"+"SRC_meta_Musical_instruments_id_treated"+"_"+today.strftime("%d%m%Y")+".pkl")